# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,14.25,58,75,2.06,CL,1706008627
1,1,stribro,49.7529,13.0041,6.03,100,99,7.11,CZ,1706008629
2,2,blackmans bay,-43.0167,147.3167,15.07,85,93,0.58,AU,1706008630
3,3,port-aux-francais,-49.3500,70.2167,7.83,98,100,8.77,TF,1706008631
4,4,waitangi,-43.9535,-176.5597,17.68,99,99,2.24,NZ,1706008632


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)
# Display the map
city_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_data_df = city_data_df[(city_data_df['Cloudiness'] == 0) & \
                                   (city_data_df['Max Temp'] >= 21) & \
                                   (city_data_df['Max Temp'] <= 27) & \
                                   (city_data_df['Wind Speed'] <= 4.5)]

# Drop any rows with null values
narrow_city_data_df = narrow_city_data_df.dropna(how='any')

# Display sample data
narrow_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,hawaiian paradise park,19.5933,-154.9731,21.69,90,0,2.57,US,1706008640
62,62,illapel,-31.6308,-71.1653,21.12,68,0,0.61,CL,1706008688
68,68,ormara,25.2088,64.6357,25.99,35,0,3.68,PK,1706008696
175,175,essaouira,31.5125,-9.7700,22.72,30,0,1.03,MA,1706008805
186,186,sohagpur,22.7000,78.2000,24.76,17,0,2.47,IN,1706008816
281,281,san luis de la loma,17.2706,-100.8939,24.20,62,0,1.73,MX,1706008916
285,285,shivpuri,25.4333,77.6500,21.02,19,0,2.93,IN,1706008922
413,413,yamethin,20.4333,96.1500,24.78,30,0,2.22,MM,1706009048
427,427,haiku-pauwela,20.9219,-156.3051,21.13,80,0,3.60,US,1706009061
462,462,shwebo,22.5667,95.7000,22.93,44,0,1.17,MM,1706009097


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_data_df[['City' , 'Country' , 'Lat' , 'Lng' ,'Humidity']]
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

C:\Users\royip\AppData\Local\Temp\ipykernel_5984\576075356.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = " "


,City,Country,Lat,Lng,Humidity,Hotel Name
12,hawaiian paradise park,US,19.5933,-154.9731,90,
62,illapel,CL,-31.6308,-71.1653,68,
68,ormara,PK,25.2088,64.6357,35,
175,essaouira,MA,31.5125,-9.7700,30,
186,sohagpur,IN,22.7000,78.2000,17,
281,san luis de la loma,MX,17.2706,-100.8939,62,
285,shivpuri,IN,25.4333,77.6500,19,
413,yamethin,MM,20.4333,96.1500,30,
427,haiku-pauwela,US,20.9219,-156.3051,80,
462,shwebo,MM,22.5667,95.7000,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    "apiKey":geoapify_key,
    "categories":categories	
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
illapel - nearest hotel: Domingo Ortiz de Rosas
ormara - nearest hotel: No hotel found
essaouira - nearest hotel: Essaouira Wind Palace
sohagpur - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
shivpuri - nearest hotel: Hotel SS
yamethin - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
shwebo - nearest hotel: Pyi Shwe Theingha
hilo - nearest hotel: Dolphin Bay Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
12,hawaiian paradise park,US,19.5933,-154.9731,90,No hotel found
62,illapel,CL,-31.6308,-71.1653,68,Domingo Ortiz de Rosas
68,ormara,PK,25.2088,64.6357,35,No hotel found
175,essaouira,MA,31.5125,-9.7700,30,Essaouira Wind Palace
186,sohagpur,IN,22.7000,78.2000,17,No hotel found
281,san luis de la loma,MX,17.2706,-100.8939,62,No hotel found
285,shivpuri,IN,25.4333,77.6500,19,Hotel SS
413,yamethin,MM,20.4333,96.1500,30,No hotel found
427,haiku-pauwela,US,20.9219,-156.3051,80,Inn At Mama's Fish House
462,shwebo,MM,22.5667,95.7000,44,Pyi Shwe Theingha


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
city_hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Hotel Name" , "Country"]
)

# Display the map
city_hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)